## set up

In [ ]:
!python3 -m pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!python3 -m pip install googlesearch-python bs4 charset-normalizer requests-html lxml_html_clean

from pathlib import Path
if not Path('./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf').exists():
    !wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf

In [1]:
from llama_cpp import Llama

# Load the model onto GPU
llama3 = Llama(
    "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    verbose=False,
    n_gpu_layers=-1,
    n_ctx=16384,    # n_ctx_train (131072) -- the full capacity
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,    
        temperature=0.5,     
    )["choices"][0]["message"]["content"]
    return _output

llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [ ]:
# Test case

test_question=''

messages = [
    {"role": "system", "content": "You are LLaMA-3.1-8B"},  
    {"role": "user", "content": test_question}, 
]

print(generate_response(llama3, messages))

## Agents

In [2]:
class LLMAgent():
    
    def __init__(self, role_description: str, task_description: str, llm:str="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description  
        self.task_description = task_description   
        self.llm = llm  
        
    def inference(self, message:str) -> str:
        if self.llm == 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF': 
            messages = [
                {"role": "system", "content": f"{self.role_description}"},  
                {"role": "user", "content": f"{self.task_description}\n{message}"}, 
            ]
            return generate_response(llama3, messages)
        else: # prepare for other models
            ...
            return ""

In [4]:
## TO DOCTOR

medical_explanation_agent = LLMAgent(
    role_description=(
        "You are an AI model interpretation specialist with expertise in machine learning applications "
        "in healthcare, particularly in autism spectrum disorder detection. Your role is to explain "
        "AI model predictions to medical professionals, helping them understand how the model processes "
        "clinical data to generate predictions. You have deep knowledge of both the technical aspects "
        "of machine learning models and clinical practice, enabling you to bridge the gap between "
        "AI technology and medical decision-making."
    ),
    
    task_description=(
        "Based on the clinical record excerpt and the IRIS model's prediction below, provide a technical "
        "explanation to medical professionals about how and why the model generated this prediction. "
        "Your explanation should:\n"
        "1. Clarify the relationship between the observed clinical features and the model's prediction\n"
        "2. Distinguish between correlation and causation, emphasizing that detected patterns may indicate "
        "association rather than direct causal relationships\n"
        "3. Explain the model's reasoning process and the significance of the retrieved clinical information\n"
        "4. Discuss potential limitations, uncertainties, or confounding factors\n"
        "5. Suggest clinical considerations for interpreting this prediction in the broader diagnostic context\n"
        "6. Be approximately 200-250 words to provide sufficient technical detail\n"
        "7. Use appropriate medical terminology while remaining clear and actionable"
    ),
)

In [3]:
# ## TO PATIENT

# explanation_agent = LLMAgent(
#     role_description=(
#         "You are a compassionate medical explanation specialist with expertise in developmental disorders, "
#         "particularly autism spectrum disorder. Your role is to explain medical findings to patients' families "
#         "in a clear, supportive manner. You strike a balance between honesty and sensitivity, avoiding overly "
#         "technical language while ensuring families understand the significance of the findings."
#     ),
#     task_description=(
#         "Based on the medical record excerpt and the model's prediction below, provide an explanation to the "
#         "patient's family about why the model has determined this prediction regarding autism. Your explanation should:\n"
#         "1. Be clear and accessible to non-medical audiences\n"
#         "2. Be sensitive to the family's potential emotional response\n"
#         "3. Avoid overly technical terminology, or explain it when necessary\n"
#         "4. Be approximately 150-200 words in length\n"
#         "5. Acknowledge any uncertainties or limitations where appropriate"
#     ),
# )

## Pipeline

In [5]:
def read_chunks(file_path):
    """
    Read chunks from file, separated by empty lines
    """
    with open(file_path, 'r') as f:
        content = f.read()
    chunks = [chunk.strip() for chunk in content.split('\n\n') if chunk.strip()]
    
    return chunks


def read_predictions(file_path):
    """
    Read predictions from file, one prediction per line
    """
    with open(file_path, 'r') as f:
        predictions = [line.strip() for line in f if line.strip()]
        
    return predictions

### one file

In [ ]:
async def generate_explanations(chunks_file, predictions_file, factor):
    """
    Generate explanations for each patient based on their chunk and prediction
    """
    chunks = read_chunks(chunks_file)
    predictions = read_predictions(predictions_file)
    
    # if len(chunks) != len(predictions):
    #     raise ValueError(f"Number of chunks ({len(chunks)}) does not match number of predictions ({len(predictions)})")
    
    with open("explanation.txt", "w") as explanation_file:
        
        # Process each patient
        for i, (chunk, prediction) in enumerate(zip(chunks, predictions)):

            is_autism = "positive" if prediction == "1" else "negative"
            
            input_text = (
                f"Factor: {factor}\n\n"
                f"Medical Record Chunk:\n{chunk}\n\n"
                f"Model Prediction: {is_autism} for autism detected"
            )
            explanation = explanation_agent.inference(input_text)
            
            # Print explanation
            print(f"\nPatient {i+1} (Prediction: {is_autism} for autism)")
            print(explanation)
            print("-" * 50)
            
            # Write to file
            explanation_file.write(f"Patient {i+1} (Prediction: {is_autism} for autism)\n\n")
            # explanation_file.write("-" * 50 + "\n")
            explanation_file.write(explanation + "\n\n")
            explanation_file.write("-" * 50 + "\n")
    
    print(f"\nFinish!!")

In [ ]:
# Main function
async def main():
    factor = "Late Talking" 
    chunks_file = "late_talking.txt"
    predictions_file = "prediction.txt"
    
    await generate_explanations(chunks_file, predictions_file, factor)

await main()

### whole folder

In [6]:
async def generate_explanations(chunks_file, predictions_file, factor, output_file):
    """
    Generate explanations for each patient based on their chunk and prediction
    """
    chunks = read_chunks(chunks_file)
    predictions = read_predictions(predictions_file)
    
    # Ensure number of chunks matches number of predictions
    if len(chunks) != len(predictions):
        raise ValueError(f"Number of chunks ({len(chunks)}) does not match number of predictions ({len(predictions)}) for {factor}")
    
    # Create factor header
    factor_header = f"\nFactor: {factor}\n{'=' * 50}\n\n"
    
    # Write factor header to console and file
    print(factor_header)
    with open(output_file, "a") as explanation_file:
        explanation_file.write(factor_header)
    
    # Process each patient
    for i, (chunk, prediction) in enumerate(zip(chunks, predictions)):

        is_autism = "Signs of autism detected" if prediction == "1" else "No signs of autism detected"
        input_text = (
            f"Factor: {factor}\n\n"
            f"Medical Record Excerpt:\n{chunk}\n\n"
            f"Model Prediction: {is_autism}"
        )
        explanation = explanation_agent.inference(input_text)
        
        patient_output = (
            f"Patient {i+1} (Prediction: {is_autism})\n"
            f"{'-' * 50}\n"
            f"{explanation}\n"
            f"{'-' * 50}\n\n"
        )

        print(patient_output)
        with open(output_file, "a") as explanation_file:
            explanation_file.write(patient_output)
    
    # Add closing separator
    closing = f"{'=' * 50}\n\n"
    with open(output_file, "a") as explanation_file:
        explanation_file.write(closing)

In [7]:
# Main function to process all factors
import os
import asyncio
import glob

async def main():
    
    # Define paths
    retrieved_chunk_dir = "retrieved_chunk"
    pred_dir = "pred"
    output_file = "explanation.txt"
    
    # Initialize or clear the output file
    with open(output_file, "w") as f:
        f.write("# Autism Factor Explanations\n\n")
    
    # Get all chunk files
    chunk_files = glob.glob(os.path.join(retrieved_chunk_dir, "*.txt"))
    
    # Process each factor
    for chunks_file in chunk_files:
        
        # Extract factor name from filename
        factor = os.path.basename(chunks_file).split('.')[0]
        
        # Construct path to corresponding prediction file
        predictions_file = os.path.join(pred_dir, f"{factor}_pred.txt")
        
        # Check if prediction file exists
        if not os.path.exists(predictions_file):
            print(f"Warning: Prediction file for {factor} not found at {predictions_file}")
            continue
        
        # Generate explanations for this factor
        try:
            await generate_explanations(chunks_file, predictions_file, factor, output_file)
            print(f"Processed factor: {factor}")
        except Exception as e:
            print(f"Error processing {factor}: {str(e)}")
    
    print(f"\nDone! All explanations have been written to {output_file}")


await main()


Factor: late_talking


Patient 1 (Prediction: Signs of autism detected)
--------------------------------------------------
I want to start by acknowledging that receiving a prediction like this can be a lot to take in, and it's completely normal to feel a range of emotions. Our model is suggesting that [NAME] may have some characteristics that are commonly seen in autism spectrum disorder, but it's essential to remember that this is just a prediction and not a definitive diagnosis.

The model has identified that [NAME] has had some challenges with speech development, which is a common area of concern for children with autism. Specifically, the model has noted that [NAME] has been late to start talking and has not consistently used words to communicate. While it's great to hear that [NAME] has made progress in physical development, the fact that he has difficulty with speech and may seem like he's not listening at times raises some red flags.

It's also important to note that the model